In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('kermans').getOrCreate()

In [3]:
df = spark.sql('select * from hack_data')

In [4]:
df.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 5 rows

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'],outputCol = 'features')

In [7]:
output = assembler.transform(df)

In [8]:
df2 = output.select('features')

In [9]:
df2.show(5)

+--------------------+
 features|
+--------------------+
[8.0,391.09,1.0,2...|
[20.0,720.99,0.0,...|
[31.0,356.32,1.0,...|
[2.0,228.08,1.0,2...|
[20.0,408.5,0.0,3...|
+--------------------+
only showing top 5 rows

In [10]:
# scalling the data
from pyspark.ml.feature import StandardScaler

In [11]:
scaler =StandardScaler(inputCol='features',outputCol='scaled_features')

In [12]:
scaled_data = scaler.fit(df2).transform(df2)

In [13]:
final_data = scaled_data.select('scaled_features')

In [14]:
from pyspark.ml.clustering import KMeans

In [15]:
kmeans = KMeans(featuresCol='scaled_features',k=3,seed=1)

In [16]:
model = kmeans.fit(final_data)

In [17]:
# within set sum of squared error
wssse = model.computeCost(final_data)
wssse

Out[ 64 ]: 434.75507308487647

In [18]:
#cluster centers
model.clusterCenters()

Out[ 65 ]: 
[array([ 1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
 5.26676612]),
 array([ 3.05623261, 2.95754486, 1.99757683, 3.2079628 , 4.49941976,
 3.26738378]),
 array([ 2.93719177, 2.88492202, 0. , 3.19938371, 4.52857793,
 3.30407351])]

In [19]:
# assigning lable to data points
result = model.transform(final_data)

In [20]:
result.show(10)

+--------------------+----------+
 scaled_features|prediction|
+--------------------+----------+
[0.56785108466505...| 0|
[1.41962771166263...| 0|
[2.20042295307707...| 0|
[0.14196277116626...| 0|
[1.41962771166263...| 0|
[0.07098138558313...| 0|
[1.27766494049636...| 0|
[1.56159048282889...| 0|
[1.06472078374697...| 0|
[0.85177662699757...| 0|
+--------------------+----------+
only showing top 10 rows

In [21]:
result.groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 88|
 2| 79|
 0| 167|
+----------+-----+

In [22]:
# seems the groups are not equal.but these should be. lets try k = 2

In [23]:
kmeans_1 = KMeans(featuresCol='scaled_features',k=2,seed=1)
model_1 = kmeans_1.fit(final_data)
result_1 = model_1.transform(final_data)

In [24]:
# now the two groups are equal
result_1.groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+